In [1]:
!pip install pymp-pypi

In [2]:
from __future__ import print_function

import pymp
import random
import numpy as np
import time
from sympy import isprime


Linear Search

In [3]:
#%%timeit
def linearsearch(arr,search_var):
  return sum(arr[i] == search_var for i in range(len(arr)))

number_to_search=5

input_array = [random.randint(1,10) for _ in range(100)]
array_len=len(input_array)


count = linearsearch(input_array,number_to_search) 

print(count)

7


In [4]:


number_to_search=5
array_len=100

ex_array = pymp.shared.array((array_len,), dtype='uint8')
for index in range(array_len):
  ex_array[index] = random.randint(1,10)

count = 0

with pymp.Parallel(4) as p:
  for index in p.range(0,array_len):
    if ex_array[index]==number_to_search:
      count+=1

print(count)

2


Check Prime Number

In [5]:
%%time
arr_len=1000

data=pymp.shared.array((arr_len,),dtype='uint8')
res=pymp.shared.array((1,),dtype='uint8')

def isPrime(n): 
	if (n <= 1) : 
		return False
	if (n <= 3) : 
		return True
	if (n % 2 == 0 or n % 3 == 0) : 
		return False
	i = 5
	while i**2 <= n: 
		if (n % i == 0 or n % (i + 2) == 0) : 
			return False
		i += 6

	return True

def check_prime(arr):
  res_s=[]
  for i in arr:
    if isPrime(i):
      res_s.append(True)
    else:
      res_s.append(False)
  return res_s



      
res_seq=check_prime(data)

CPU times: user 2.56 ms, sys: 830 µs, total: 3.39 ms
Wall time: 3.01 ms


In [6]:


for i in range(arr_len):
  data[i]=random.randint(0,9)

with pymp.Parallel(4) as p:
  for index in p.range(arr_len):
    if isPrime(data[index])==True:
      res=np.append(res,np.array([True]))
    else:
      res=np.append(res,np.array([False]))

Matrix Multiplication

In [7]:

size = 99
mat1 = np.random.randint(1,100000,(size,size))
mat2 = np.random.randint(1,100000,(size,size))


res = [[0 for _ in range(mat1.shape[0])] for _ in range(mat2.shape[0])]  

for i in range(len(mat1)): 
    for j in range(len(mat2[0])): 
        for k in range(len(mat2)): 

            res[i][j] += mat1[i][k] * mat2[k][j] 


In [8]:

import itertools
ex_array = pymp.shared.array((mat1.shape[0],mat1.shape[0]), dtype='uint8')
for index, j in itertools.product(range(mat1.shape[0]), range(mat2.shape[0])):
  ex_array[index][j] = 0
start = time.time()

# parallelize only the outer layer of loop
with pymp.Parallel(4) as p:
  for i in p.range(len(mat1)):
    for j in range(len(mat2[0])): 
      for k in range(len(mat2)): 
        ex_array[i][j] += mat1[i][k] * mat2[k][j]
print(time.time()-start)

start = time.time()

with pymp.Parallel(4) as p:
  for i in p.range(len(mat1)): 
    for j in p.range(len(mat2[0])): 
      for k in range(len(mat2)): 
        ex_array[i][j] += mat1[i][k] * mat2[k][j]
print(time.time()-start)
start = time.time()

with pymp.Parallel(4) as p:
  for i in p.range(len(mat1)): 
    for j in p.range(len(mat2[0])): 
      for k in p.range(len(mat2)): 
        ex_array[i][j] += mat1[i][k] * mat2[k][j]
print(time.time()-start)

# print(ex_array.shape)
# with pymp.Parallel(4) as p:
#   for index in p.range(mat1.shape[0]):
#     if ex_array[index]==number_to_search:
#       count+=1

# Complete the pymp parallel code here
# If there are nested for loops, parallelize 
# 1. Only outer loop 
# 2. Only inner loop and 
# 3. All loops. 
# Finally, analyze the time taken by different parallelization settings.

1.2375049591064453
0.37471699714660645
0.13127613067626953


Check Prime Number in PowerSet


In [9]:
def powerset(s):
  x = len(s)
  p_list=[]
  for i in range(1 << x):
    p_list.append([s[j] for j in range(x) if (i & (1 << j))])
  return p_list
powerset_list=powerset([i for i in range(1,25)])


In [10]:
def checkprime(arr):
  arrsum=0
  res=[]
  for i in arr:
      res.append(isprime(sum(i)))
  return res





In [11]:
start=time.time()
single_process_res=checkprime(powerset_list)
print('Single process takes ',time.time()-start)


Single process takes  21.526139974594116


In [13]:
len_list=len(powerset_list)
ex_array = pymp.shared.array((len_list), dtype='uint8')
for index in range(len_list):
    ex_array[index] = False
    
start=time.time()

with pymp.Parallel(4) as p:
        # ex_array[i]=isprime(sum(powerset_list[i]))
    for i in p.range(len_list):
        if isprime(sum(powerset_list[i])):
            ex_array[i]=True
        else:
            ex_array[i]=False
            

print('Parallel Process takes ',time.time()-start)


Parallel Process takes  10.542746782302856
